In [203]:
from os import listdir
from os.path import isfile, join
import json
import math
import ast
import openpyxl
import numpy as np
import requests
import time


In [204]:
import pandas as pd
import pickle

from mdutils.mdutils import MdUtils
from mdutils import Html



In [205]:
token = "INSERT YOUR API TOKEN HERE"
headers = {"Authorization": "Bearer " + token, }

endpoint = "https://europe-west1-tagr-prod.cloudfunctions.net/addToNodeV2"

endpoint2 = 'http://127.0.0.1:5001/emulator/europe-west1/addToNodeV2'

In [206]:
testdata = {
  'nodes': [
    {
      'name': 'The **Hobbit**',
      'description': 'A book by J.R.R. Tolkien',
      'supertags': [{'id':'iD4KFNucdt'}],
      'children': []
    }
  ]
}

In [207]:
r = requests.post(endpoint,json=testdata, headers=headers)


In [208]:
r.status_code

200

In [209]:
r.json()

{'children': [{'nodeId': 'kKIU3-wChbKX', 'name': 'The <b>Hobbit</b>', 'description': 'A book by J.R.R. Tolkien', 'type': 'node'}]}

## Configuration


### Configure all the Supertags inside Tana, and add the IDs to the list below

Inside Tana, with the cursor inside a Supertag object, run command: "Get API Schema" to receive the schema information with the IDs of the Supertag and its fields.

In [210]:
author_tag = "xTCjqfx_Am"
book_tag = "UDLQyNUHvs"
concept_tag = "Z_V_nY1BWy"
highlight_tag = "E0k3Hh83r6"
text_tag = "LJLkveJAnqe-"


author_loc = "PRoekPc17f"
concept_loc = "KhJdLz82z4"
highlight_loc = "5_NVFzdq3f"

highlight_author = 'iXptO18LlB'
highlight_text = 'MXWU0_jcMc'
highlight_book = 'musVv26DvX'
highlight_concepts = 'uj8YnZxp0l'
nearest_highlights = '_xc5TDVW9o'

highlight_same_field ='yIVdaP1LlR'
highlight_diff_field =  '8n1Sg3j1nN'

concept_definition = 'iR3DuDRgcH'
concept_links_field = '1iLkAFIwfS'
related_concepts_field = 'of3GlQaNGM'

book_author = 'VmnhTQRb5d'

### Example of Supertag API Schema:

In [211]:
"""

/* Highlight Author */
      type: 'field';
      attributeId: 'iXptO18LlB';
      children: Node[];
    },
    {
      /* Highlight Texts */
      type: 'field';
      attributeId: 'MXWU0_jcMc';
      children: Node[];
    },
    {
      /* Highlight's Concepts */
      type: 'field';
      attributeId: 'uj8YnZxp0l';
      children: Node[];
    },
    {
      /* Nearest Highlights */
      type: 'field';
      attributeId: '_xc5TDVW9o';
      children: Node[];





 /* Concept Definition */
      type: 'field';
      attributeId: 'iR3DuDRgcH';
      children: Node[];
    },
    {
      /* Concept Links */
      type: 'field';
      attributeId: '1iLkAFIwfS';
      children: Node[];
    },
    {
      /* Related Concept */
      type: 'field';
      attributeId: 'of3GlQaNGM';
      children: Node[];



/* Author Name */
      type: 'field';
      attributeId: 'LRtFpsHk0Z';
      children: Node[];
    },
    {
      /* Author's Concepts */
      type: 'field';
      attributeId: 'uEiI1pLjF9';
      children: Node[];



Same Author Highlights */
      type: 'field';
      attributeId: 'yIVdaP1LlR';
      children: Node[];
    },
    {
      /* Diff Author Highlights */
      type: 'field';
      attributeId: '8n1Sg3j1nN';
      children: Node[];
    },

"""

"\n\n/* Highlight Author */\n      type: 'field';\n      attributeId: 'iXptO18LlB';\n      children: Node[];\n    },\n    {\n      /* Highlight Texts */\n      type: 'field';\n      attributeId: 'MXWU0_jcMc';\n      children: Node[];\n    },\n    {\n      /* Highlight's Concepts */\n      type: 'field';\n      attributeId: 'uj8YnZxp0l';\n      children: Node[];\n    },\n    {\n      /* Nearest Highlights */\n      type: 'field';\n      attributeId: '_xc5TDVW9o';\n      children: Node[];\n\n\n\n\n\n /* Concept Definition */\n      type: 'field';\n      attributeId: 'iR3DuDRgcH';\n      children: Node[];\n    },\n    {\n      /* Concept Links */\n      type: 'field';\n      attributeId: '1iLkAFIwfS';\n      children: Node[];\n    },\n    {\n      /* Related Concept */\n      type: 'field';\n      attributeId: 'of3GlQaNGM';\n      children: Node[];\n\n\n\n/* Author Name */\n      type: 'field';\n      attributeId: 'LRtFpsHk0Z';\n      children: Node[];\n    },\n    {\n      /* Author's Co

## Parameters

In [212]:
#CHANGE BASE DIRECTORY ACCORDINGLY

base_dir = join("Examples", "Example Obsidian Vault 8 Books")

dirr = join(base_dir, "05_model_input/")

md_dir = join(base_dir, "Texts_Vault")


num_results = 2

concept_count_for_author = 10

dbpedia_types = {'Person': True}

#"http://dbpedia.org/ontology/Person"


conceptsdir = join(base_dir, "04_feature")



## Read Files

In [213]:
raw_text_df = pd.read_csv(dirr+"text_df_final.csv", index_col=0)

text_df = pd.read_csv(dirr+"text_df_final.csv", index_col=0)

#text_df = pd.read_csv("data/01_raw/text_df.csv", index_col=0)

mentions_df = pd.read_csv(dirr+"mentions_df_final.csv")


mentions_df.sort_values("Start_pos", ascending=False, inplace=True)


relatedness_semantic_df = pd.read_csv(dirr+"relatedness_df.csv")
relatedness_semantic_df.index = relatedness_semantic_df['Text_ID']

relatedness_concepts_df = pd.read_csv(dirr+"relatedness_concepts_df.csv", index_col=0)

relatedness_concepts_df_repeat = pd.read_csv(dirr+"relatedness_concepts_df_repeat.csv", index_col=0)



with open(dirr+"conceptnet_to_dbpedia.json", "r") as f:
        conceptnet_to_dbpedia =  json.load(f)


with open(dirr+"concepts_list_final.json", "r") as f:
        concepts_list = json.load(f)


def add_title_column(text_df):
	title_column = []
	count_dict = {}
	for name in text_df['Father_name'].unique():
		count_dict[name.split(":")[0]] = 1

	for row in text_df.itertuples():
		name = row[2].split(":")[0]
		title = name + " Note " + str(count_dict[name])
		title_column.append(title)
		count_dict[name] += 1
	text_df['Title'] = title_column
	return text_df

text_df = add_title_column(text_df)


In [214]:
len(concepts_list)

152

In [215]:
conceptsdir = join(base_dir, "04_feature")

with open(join(join(base_dir, "03_primary"),"concepts_data_list.json"), "r") as f:
        concepts_data_list = json.load(f)

with open(join(conceptsdir,"conceptnet_distances_dict.json"), "r") as f:
        conceptnet_distances_dict = json.load(f)

with open(join(conceptsdir,"conceptnet_relations_dict.json"), "r") as f:
        conceptnet_relations_dict = json.load(f)



new_concepts_data_list = []
for data in concepts_data_list:
	if type(data) != type([0]):
		print("Error")
		print(data)
	else:
		new_concepts_data_list.append(data)


column_names = ["DBpedia URL", "Name", "Abstract", "Image", "DBpedia Types", "Subject FROM", "Related Entities FROM", "Related Entities OF", "Subject OF", "Narrower Categories", "Broader Categories"] 

concept_df = pd.DataFrame(new_concepts_data_list, columns=column_names)

concepts_database = concept_df[concept_df["DBpedia URL"].isin(concepts_list)]


relations_dict_column = []
distances_dict_column = []

for row in concepts_database.itertuples():

	concept_id = row[1]

	distances_dict = conceptnet_distances_dict[concept_id]

	relations_dict = conceptnet_relations_dict[concept_id]

	distances_dict_column.append(distances_dict)

	relations_dict_column.append(relations_dict)

concepts_database_enhanced = concepts_database.copy()

concepts_database_enhanced['ConceptNet_Distances'] = distances_dict_column

concepts_database_enhanced['ConceptNet_Relations'] = relations_dict_column

concepts_df = concepts_database_enhanced

concepts_list = list(concepts_df["DBpedia URL"])
print(len(concepts_list))

concepts_df = concepts_df.set_index("DBpedia URL")


127


## Author Data

In [216]:
author_texts_dict = {}

author_list = text_df['Author_name'].unique()

for author in author_list:
	#Related Concepts:
	author_texts_dict[author] = []

for index, row in text_df.iterrows():
	text_author = row['Author_name']
	text_id = row["Text_ID"]
	author_texts_dict[text_author].append(str(text_id))


	#Related Authors


In [217]:



author_relatedness = {}
i = 0

max_relatedness_list = []
avg_relatedness_list = []

for author1 in author_list:
	j = 0
	author1_texts = author_texts_dict[author1]
	num_texts = len(author1_texts)
	row_max = []
	row_avg = []
	for author2 in author_list:
		if author1 == author2:
			final_average_relatedness = 1
			max_relatedness = 1
		else:
			author2_texts = author_texts_dict[author2]
			total_relatedness = 0
			max_relatedness = 0

			for text_id in author1_texts:
				average_relatedness = relatedness_semantic_df.loc[int(text_id)][author2_texts].mean()
				if average_relatedness > max_relatedness:
					max_relatedness = average_relatedness
				total_relatedness+=average_relatedness
			final_average_relatedness = total_relatedness/num_texts
		row_max.append(max_relatedness)
		
		row_avg.append(final_average_relatedness)

	max_relatedness_list.append(row_max)
	avg_relatedness_list.append(row_avg)


avg_relatedness_df = pd.DataFrame(avg_relatedness_list, index=author_list, columns=author_list)




In [218]:
mentions_df.head(2)

,Text_ID,Father_ID,Father_Name,Author_name,Source,Mention text,Start_pos,End_pos,Entity Name,DBpedia URL,Confidence,Description,Semantic Categories,DBpedia Types
645,590635937,31828755,"The PARA Method: Simplify, Organise and Master...",Tiago Forte,Dandelion,ideas,1363,1368,Idea,http://dbpedia.org/resource/Idea,0.6928,"In common usage and in philosophy, ideas are t...","['A priori', 'Abstraction', 'Cognition', 'Conc...",[]
649,590635937,31828755,"The PARA Method: Simplify, Organise and Master...",Tiago Forte,Dandelion,time,1309,1313,Time,http://dbpedia.org/resource/Time,0.7034,Time is the continued sequence of existence an...,"['Concepts in aesthetics', 'Concepts in episte...",[]
566,583516125,31385202,The Knowledge Illusion: The myth of individual...,Steven Sloman and Philip Fernbach,Dandelion,information,1267,1278,Information,http://dbpedia.org/resource/Information,0.6857,Information is an abstract concept that refers...,"['Concepts in metaphysics', 'Information', 'In...",[]
563,583516125,31385202,The Knowledge Illusion: The myth of individual...,Steven Sloman and Philip Fernbach,Dandelion,abstract,1258,1266,Abstraction,http://dbpedia.org/resource/Abstraction,0.6618,Abstraction is a conceptual process wherein ge...,"['Abstraction', 'Concepts in epistemology', 'C...",[]
562,583516125,31385202,The Knowledge Illusion: The myth of individual...,Steven Sloman and Philip Fernbach,Dandelion,abstract,1232,1240,Abstraction,http://dbpedia.org/resource/Abstraction,0.6618,Abstraction is a conceptual process wherein ge...,"['Abstraction', 'Concepts in epistemology', 'C...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,514356015,26811735,"Why Information Grows: The Evolution of Order,...",Cesar Hidalgo,Dandelion,Humans,0,6,Human,http://dbpedia.org/resource/Human,0.6855,"Humans, or modern humans (Homo sapiens), are t...","['Apex predators', 'Articles containing video ...",[]
7,110306480,6297743,How to Take Smart Notes: One Simple Technique ...,Sönke Ahrens,Dandelion,Writing,0,7,Writing,http://dbpedia.org/resource/Writing,0.7299,Writing is a cognitive and social activity inv...,"['Behavior', 'Human communication', 'Language'...",[]
505,583516119,31385202,The Knowledge Illusion: The myth of individual...,Steven Sloman and Philip Fernbach,Dandelion,Memory,0,6,Memory,http://dbpedia.org/resource/Memory,0.7702,Memory is the faculty of the mind by which dat...,"['Memory', 'Mental processes', 'Neuropsycholog...",[]
493,583516118,31385202,The Knowledge Illusion: The myth of individual...,Steven Sloman and Philip Fernbach,Dandelion,Thought,0,7,Thought,http://dbpedia.org/resource/Thought,0.7719,"In their most common sense, the terms thought ...","['Abstraction', 'Cognition', 'Cognitive psycho...",[]


In [219]:

author_concept_count = {}

for author in author_list:

	author_concept_count[author] = {}

for index, row in mentions_df.iterrows():
	mention_author = row['Author_name']
	concept = row["Entity Name"]

	if concept not in author_concept_count[mention_author].keys():

		author_concept_count[mention_author][concept] = 1
	else:
		author_concept_count[mention_author][concept] += 1


author_concept_dict = {}
for author in author_list:
	author_concepts = sorted(author_concept_count[author].items(), key=lambda item: item[1], reverse=True)
	author_concept_dict[author] = author_concepts

## Manipulations Text Nodes

### Add Text Relationships

In [221]:
relatedness_semantic_df.head(3)

def create_related_texts_dataframe(semantic_similarities, num_results):
	
	recommendations_df_list = []

	for index, row in semantic_similarities.iterrows():

		idd = row["Text_ID"]
		#print(idd)
		query_author = row["Author_name"]

		column_name = str(idd)

		query_column_names = ["Text_ID", "Author_name", column_name]

		query_df = semantic_similarities[query_column_names].copy().sort_values(column_name, ascending=False).reset_index(drop=True)

		same_author_key_list = []
		different_author_key_list = []

		first = True
		for row2 in query_df.itertuples():
			key = row2[1]
			author_name = row2[2]
			score = row2[3]
			
			#distance = 0

			# Append a distance em algum lugar ↓

			if author_name == query_author and first == False:
				if len(same_author_key_list) < num_results:
					same_author_key_list.append({key:score})
			elif first == False:
				if len(different_author_key_list) < num_results:
					different_author_key_list.append({key:score})
			first = False
			
		recommendations_df_list.append([idd, same_author_key_list, different_author_key_list])
		
	semantic_recommendations_df = pd.DataFrame(recommendations_df_list, columns=["Text_ID", "Same Author Recs", "Diff Author Recs"])
	semantic_recommendations_df.set_index("Text_ID", inplace=True)

	return semantic_recommendations_df

In [222]:
semantic_recommendations = create_related_texts_dataframe(relatedness_semantic_df, num_results=num_results)

semantic_recommendations

,Same Author Recs,Diff Author Recs
Text_ID,,
110306476,"[{290432993: 0.91128}, {290432995: 0.5151}]","[{406891963: 0.51646}, {336303820: 0.50196}]"
110306477,"[{290432997: 0.49518}, {290432999: 0.47503}]","[{590584777: 0.48987}, {590635942: 0.45945}]"
110306479,"[{290432993: 0.55997}, {290432999: 0.49657}]","[{583516121: 0.57397}, {336303818: 0.46484}]"
110306480,"[{290432996: 0.51655}, {290433000: 0.446}]","[{336303816: 0.55601}, {590605099: 0.55483}]"
110306481,"[{376444506: 0.50613}, {376444505: 0.47604}]","[{463559130: 0.59345}, {583516114: 0.58909}]"
...,...,...
591548161,"[{591548159: 0.73799}, {591548154: 0.70139}]","[{376433767: 0.53204}, {583516114: 0.53143}]"
591548162,"[{547332530: 0.63187}, {547332529: 0.63018}]","[{583516124: 0.53747}, {583516121: 0.5209}]"
591548163,"[{591548165: 0.57666}, {514356011: 0.52415}]","[{583516121: 0.56975}, {376444506: 0.55311}]"


In [223]:

concept_based_recommendations = create_related_texts_dataframe(relatedness_concepts_df, num_results=num_results)

concept_based_recommendations

,Same Author Recs,Diff Author Recs
Text_ID,,
110306476,"[{290432993: 0.4918069564908715}, {110306479: ...","[{590605099: 0.3904362057500617}, {590605101: ..."
110306479,"[{290432993: 0.4140740508369426}, {110306476: ...","[{591548151: 0.30280276816609}, {583516118: 0...."
110306480,"[{290433001: 0.6229012414727946}, {376444509: ...","[{336303818: 0.7828186265910537}, {496905538: ..."
290432993,"[{110306476: 0.4918069564908715}, {110306479: ...","[{590605099: 0.3759842150471769}, {590605101: ..."
290432995,"[{110306480: 0.5036375726620896}, {376444509: ...","[{336303818: 0.5471934385250757}, {496905538: ..."
...,...,...
591548161,"[{591548159: 0.4989619955033399}, {514356017: ...","[{376444501: 0.4977929051503558}, {376433767: ..."
591548162,"[{591548163: 0.5361312377537832}, {514356017: ...","[{406891963: 0.6024259551481475}, {336303815: ..."
591548163,"[{547332524: 0.6101140221580419}, {591548165: ...","[{376444509: 0.7996482294246017}, {590635944: ..."


# Build Knowledge Repository → Tana

## Create Text Nodes

In [224]:
def get_title_for_highlight_id(id_dict):
	id = list(id_dict.keys())[0]
	try:
		title = text_df.loc[id, "Title"]
	except:
		title = ""
	
	return title, round(id_dict[id], 3)

	

## Add Concepts Links

In [225]:
concepts_df.head(2)

,Name,Abstract,Image,DBpedia Types,Subject FROM,Related Entities FROM,Related Entities OF,Subject OF,Narrower Categories,Broader Categories,ConceptNet_Distances,ConceptNet_Relations
DBpedia URL,,,,,,,,,,,,
http://dbpedia.org/resource/Personal_knowledge_management,Personal knowledge management,Personal knowledge management (PKM) is a proce...,,[http://dbpedia.org/class/yago/WikicatInterdis...,[http://dbpedia.org/resource/Category:Knowledg...,[http://dbpedia.org/resource/Pattern_recogniti...,"[http://dbpedia.org/resource/Zettelkasten, htt...",[],[],[],"{'http://dbpedia.org/resource/Semantics': 0.0,...","{'/r/RelatedTo': [], '/r/IsA': [], '/r/HasCont..."
http://dbpedia.org/resource/Thought,Thought,"In their most common sense, the terms thought ...",http://commons.wikimedia.org/wiki/Special:File...,[http://www.w3.org/2002/07/owl#Thing],[http://dbpedia.org/resource/Category:Intellig...,[http://dbpedia.org/resource/Outline_of_though...,[http://dbpedia.org/resource/Thinking_Skills_a...,[],[],[],{'http://dbpedia.org/resource/Discipline': -0....,"{'/r/RelatedTo': ['/c/en/think', '/c/en/idea',..."
http://dbpedia.org/resource/Mind,Mind,The mind is the set of faculties responsible f...,http://commons.wikimedia.org/wiki/Special:File...,[http://www.w3.org/2002/07/owl#Thing],[http://dbpedia.org/resource/Category:A_priori...,[http://dbpedia.org/resource/Internalism_and_e...,"[http://dbpedia.org/resource/Katalemwa, http:/...",[],[],[],{'http://dbpedia.org/resource/Discipline': 0.0...,"{'/r/RelatedTo': ['/c/en/brain', '/c/en/thinki..."
http://dbpedia.org/resource/Brain,Brain,A brain is an organ that serves as the center ...,http://commons.wikimedia.org/wiki/Special:File...,"[http://www.w3.org/2002/07/owl#Thing, http://d...",[http://dbpedia.org/resource/Category:Human_an...,"[http://dbpedia.org/resource/Fatty_acid, http:...","[http://dbpedia.org/resource/Piotr_Popik, http...",[],[],[],{'http://dbpedia.org/resource/Discipline': -0....,"{'/r/RelatedTo': ['/c/en/head', '/c/en/attenti..."
http://dbpedia.org/resource/Internet,Internet,The Internet (or internet) is the global syste...,http://commons.wikimedia.org/wiki/Special:File...,"[http://www.w3.org/2002/07/owl#Thing, http://d...",[http://dbpedia.org/resource/Category:Internet...,"[http://dbpedia.org/resource/London, http://db...",[http://dbpedia.org/resource/KTU_radio_Gaudeam...,[],[],[],{'http://dbpedia.org/resource/Semantics': 0.05...,"{'/r/RelatedTo': ['/c/en/intranet', '/c/en/net..."


In [226]:
#concepts_df.head(2)
 
concept_links = {}

conceptnet_keys = list(conceptnet_to_dbpedia.keys())

for concept_url in concepts_list:
	concept_links[concept_url] = {'link_from': [], 'link_to': [], 'both_links': [], 'shared_subject':{}, 'conceptnet_relations': []}
	#print(concept_url)
	#print(concepts_df_dbpedia_index.loc[concept_url, "Name"])

for concept_url in concepts_list:

	concepts_linked_from_this_concept = concepts_df.loc[concept_url, "Related Entities FROM"]
	this_concept_is_linked_from = concepts_df.loc[concept_url, "Related Entities OF"]
	subject_list = concepts_df.loc[concept_url, "Subject FROM"]

	conceptnet_dict = concepts_df.loc[concept_url, "ConceptNet_Relations"] #ast.literal_eval(concepts_df.loc[concept_url, "ConceptNet_Relations"])

	for relation in conceptnet_dict.keys():
		related_concepts = conceptnet_dict[relation]
		for related_concept in related_concepts:
			if related_concept in conceptnet_keys and conceptnet_to_dbpedia[related_concept] != concept_url:
				concept_links[concept_url]['conceptnet_relations'].append([relation, conceptnet_to_dbpedia[related_concept]])

	for other_concept in concepts_list:

		

		if other_concept in concepts_linked_from_this_concept:
			concept_links[other_concept]['link_from'].append(concept_url)

		if other_concept in this_concept_is_linked_from:
			concept_links[other_concept]['link_to'].append(concept_url)

		if other_concept in concepts_linked_from_this_concept and other_concept in this_concept_is_linked_from:
			concept_links[other_concept]['both_links'].append(concept_url)

		other_subject_list = concepts_df.loc[other_concept, "Subject FROM"]
		if concept_url != other_concept:
			for subject in other_subject_list:

				if subject in subject_list:

					if concept_url in list(concept_links[other_concept]['shared_subject'].keys()):

						concept_links[other_concept]['shared_subject'][concept_url].append(subject)
					else: 
						concept_links[other_concept]['shared_subject'][concept_url]= [subject]
	


## Execute Concept Nodes

In [227]:
#def create_concept_md_file(name, url, abstract, image_url, concept_links):

concept_id_dict = {}

concept_post_request = {'nodes':[]}

for concept_url in concepts_list:

	print(concept_url)
	
	url = concept_url
	name = concepts_df.loc[concept_url, "Name"]
	wikiurl = "https://en.wikipedia.org/wiki/" + "_".join(name.split(" "))
	image_url = concepts_df.loc[concept_url, "Image"]
	abstract = concepts_df.loc[concept_url, "Abstract"]
	
	title = name.title()
	
	if type(image_url) ==type(" "): # image_url != "":
		image = "![" + str(name) + "](" + image_url + ")"



	used_concepts = []
	concept_relations_tana_field = []

	relation_list = ["/r/Causes",
		"/r/CapableOf",
		"/r/MotivatedByGoal",
		"/r/HasPrerequisite",
		"/r/Synonym",
		"/r/SimilarTo",
			"/r/IsA",
			"/r/RelatedTo",
			"/r/Antonym",
			"/r/DistinctFrom",
				"/r/HasContext",
				"/r/HasProperty",
				"/r/PartOf",
				"/r/UsedFor",]

	conceptnet_relations = concept_links[url]['conceptnet_relations']


	used_concept_list_dict = []


	#concept_relations_builder = {}

	for relation_type in relation_list:
		concepts_by_relation = []
		for pair in conceptnet_relations:
			relation = pair[0]
			concept = pair[1]
			if concept in concepts_list:
				concept_name = concepts_df.loc[concept, "Name"]
				if relation_type == relation:

					if concept_name not in used_concepts:
						#concept_relations_tana_field.append({'name': relation.split("r/")[1] + " " + concept_name})
						used_concepts.append(concept_name)
					
						concepts_by_relation.append(concept_name)
						

						

		if len(concepts_by_relation) > 0:
			tana_related_concepts = []
			for related_concept in concepts_by_relation:
				if related_concept in concept_id_dict.keys():
					tana_related_concepts.append({'id':concept_id_dict[related_concept], 'dataType':'reference'})
				else:
					tana_related_concepts.append({'name':related_concept})

			concept_relations_tana_field.append({'name': relation_type.split("r/")[1], 'children': tana_related_concepts})


	count=0	
	tana_wiki_concepts = []
	for concept_url in concept_links[url]['both_links']:
		concept_name = concepts_df.loc[concept_url, "Name"]
		if concept_url != url and count < 5 and concept_name not in used_concepts:
			count+=1
			#concept_relations_tana_field.append({'name': "HasWikiLink " + concept_name})
			used_concepts.append(concept_name)
			if concept_name in concept_id_dict.keys():
				tana_wiki_concepts.append({'id':concept_id_dict[concept_name], 'dataType':'reference'})
			else:
				tana_wiki_concepts.append({'name':concept_name})
	if len(tana_wiki_concepts) > 0:
		concept_relations_tana_field.append({'name': 'HasWikiLink', 'children': tana_wiki_concepts})

	
	#used_concept_list_dict = []

	#for concept_used in concept_relations_tana_field:
		#if concept_used in list(concept_id_dict.keys()):
		#current_concept = concept_used
		#used_concept_list_dict.append({'id': concept_id_dict[concept_used], 'dataType':'reference'})
		#used_concept_list_dict.append({'name': current_concept})
	
	#print(len(concept_relations_tana_field))
	print(concept_relations_tana_field)

	if len(concept_relations_tana_field) <= 0:
		print("NO Concepts Referenced")
		concepts_fields = [{
				'type': 'field',
				'attributeId': concept_definition,
				'children': [
					{
					'name': abstract
					},
					]
				},
				{
				'type': 'field',
				'attributeId': concept_links_field,
				'children': [
					{
					'name': "Wikipedia URL: " + wikiurl
					},
					{
					'name': "DBpedia URL: " + url
					}
				]}
				
				]
	else:
		print("Concepts Referenced")
		concepts_fields = [
			{
				'type': 'field',
				'attributeId': concept_definition,
				'children': [
					{
						'name': abstract
					},
					]
			},
			{
				'type': 'field',
				'attributeId': concept_links_field,
				'children': [
					{
					'name': "Wikipedia URL: " + wikiurl
					},
					{
					'name': "DBpedia URL: " + url
					}
					]
			},
			{
				'type': 'field',
				'attributeId': related_concepts_field,
				'children': concept_relations_tana_field
			}
			]
			
	concept_data = {
			'name': title.replace("(", "- ").replace(")",""),
			'supertags': [{'id':concept_tag}],
			'children': concepts_fields
			}
			
			
	
	#print(concept_data)
	concept_post_request['nodes'].append(concept_data)

	r = requests.post(endpoint, json={'nodes':[concept_data]}, headers=headers)

	posted = r.json()

	print(posted)

	if 'children' in posted.keys():
		print(posted['children'][0]['nodeId'])
		concept_id_dict[name] = posted['children'][0]['nodeId']

	time.sleep(0.5)

#print(concept_post_request)




"""
,
{
'type': 'field',
'attributeId': concept_links_field,
'children': [
	{
	'name': "Wikipedia URL: " + wikiurl
	},
	{
	'name': "DBpedia URL: " + url
	}
]}
"""

http://dbpedia.org/resource/Personal_knowledge_management
[{'name': 'HasWikiLink', 'children': [{'name': 'Commonplace book'}]}]
Concepts Referenced
{'children': [{'nodeId': '1nZrzCyBhZDP', 'name': 'Personal Knowledge Management', 'type': 'node', 'children': [{'nodeId': 'TkoC00hKCVpr', 'name': 'Personal knowledge management (PKM) is a process of collecting information that a person uses to gather, classify, store, search, retrieve and share knowledge in their daily activities and the way in which these processes support work activities. It is a response to the idea that knowledge workers need to be responsible for their own growth and learning. It is a bottom-up approach to knowledge management (KM).', 'type': 'node'}, {'nodeId': 'GowWi648Nm6B', 'name': 'Wikipedia URL: <a href="https://en.wikipedia.org/wiki/Personal_knowledge_management">https://en.wikipedia.org/wiki/Personal_knowledge_management</a>', 'type': 'node'}, {'nodeId': 'IJsF6u9pvjzn', 'name': 'DBpedia URL: <a href="http://dbp

'\n,\n{\n\'type\': \'field\',\n\'attributeId\': concept_links_field,\n\'children\': [\n\t{\n\t\'name\': "Wikipedia URL: " + wikiurl\n\t},\n\t{\n\t\'name\': "DBpedia URL: " + url\n\t}\n]}\n'

## Execute Text Nodes

In [228]:
print(active_concepts)

['Person', 'Knowledge', 'Learning', 'Experience', 'Nervous system', 'Human']


In [229]:
concept_id_dict.keys()

dict_keys(['Personal knowledge management', 'Thought', 'Mind', 'Brain', 'Internet', 'Information', 'System', 'Experience', 'Scientific method', 'Communication', 'Knowledge', 'Research', 'Decision-making', 'Philosophy', 'Writing', 'Employment', 'Social media', 'Problem solving', 'Wealth', 'Person', 'Value (ethics and social sciences)', 'Productivity', 'Time', 'Meaning of life', 'Action (philosophy)', 'Life', 'Truth', 'Interpersonal relationship', 'Learning', 'Reason', 'Visual perception', 'Idea', 'Metaphor', 'Individual', 'Understanding', 'Human', 'Nature', 'Creativity', 'Entropy', 'Intuition', 'Diary', 'Principle', 'Reality', 'Evolution', 'Skill', 'Causality', 'Stress (biology)', 'Past', 'Tool', 'Holism', 'Energy', 'Object (philosophy)', 'Time management', 'Fact', 'Point of view (philosophy)', 'Ethics', 'Mindset', 'Economic system', 'Notebook', 'Neoplatonism', 'Theory of forms', 'DNA', 'Universe', 'Sense', 'Pattern', 'Matter', 'Society', 'Discipline', 'Human body', 'Reading', 'Imaginat

In [230]:
print(highlight_author)

iXptO18LlB


In [231]:
#mentions_df.head(2)
#text_df.head(3)

count_dict = {}
author_list = []

track = 0


author_id_dict = {}

book_id_dict = {}

highlight_id_dict = {}


for row in text_df.itertuples():
	track += 1
	if track > -1:
	
		highlight_id = row[0]
		book = row[2].split(":")[0]
		author = row[3]
		text = row[4]
		#notes = row[5]
		title = row[6]


		if author not in list(author_list):
			author_list.append(author)
			num_texts_from_author = len(author_texts_dict[author])
			num_concepts = len(author_concept_dict[author])

			for concept_tuple in author_concept_dict[author][0:concept_count_for_author]:
				concept = concept_tuple[0]
				counts = concept_tuple[1]

			related_authors = list(avg_relatedness_df.sort_values(by=author, ascending=False).index)

			for text_id in author_texts_dict[author]:
				title_text = text_df.loc[int(text_id)]["Title"]

			print(author)

			author_data = {
				'nodes': [
				{
				'name': author,
				'supertags': [{'id':author_tag}],
				'children': [
					{
					'type': 'field',
					'attributeId': 'LRtFpsHk0Z',
					'children': [
					{
						'name': author
					}]
					}
				]
				}
				]
			}
			
			r = requests.post(endpoint, json=author_data, headers=headers)
			
			posted = r.json()
			
			#author_node_id = posted['children'][0]['nodeId']
			
			author_id_dict[author] = posted['children'][0]['nodeId']
			

			print(posted)

			print(posted['children'][0]['nodeId'])
			#author_posted = posted
		time.sleep(1)
		if book not in list(count_dict.keys()):
			count_dict[book] = 1
			

			book_data = {
				'nodes': [
				{
				'name': book,
				'supertags': [{'id':book_tag}],
				'children': [
					{
					'type': 'field',
					'attributeId': book_author,
					'children': [
					{	
						'id' : author_id_dict[author],
						'dataType': 'reference'
					}]
					}
				]
				}]
			}
			
			print("POSTING... " + book)


			print(book_data)

		
			r = requests.post(endpoint, json=book_data, headers=headers)
		
		
			posted = r.json()
		
			print(posted)

			if 'children' in posted.keys():
				book_id_dict[book] = posted['children'][0]['nodeId']



		time.sleep(1)


		tana_related_highliights = []
		rank = 1
		for hid in semantic_recommendations.loc[highlight_id, "Same Author Recs"]:
			print(hid)
			title2, score = get_title_for_highlight_id(hid)
			atext = "- sameauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
			rank +=1

			print("TESTEEE")
			print("NAME: ", title2)

			if list(hid.keys())[0] in highlight_id_dict.keys():
				tana_related_highliights.append({'id':highlight_id_dict[list(hid.keys())[0]], 'dataType':'reference'})
				print("TESTEEE YEAAAAHHHHH")
			else:
				tana_related_highliights.append({'name':title2})
				print("TESTEEE NOOOOO")

		
		rank = 1
		for hid in semantic_recommendations.loc[highlight_id, "Diff Author Recs"]:
			title2, score = get_title_for_highlight_id(hid)
			atext = "- diffauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
			rank +=1
	
			if list(hid.keys())[0] in highlight_id_dict.keys():
				tana_related_highliights.append({'id':highlight_id_dict[list(hid.keys())[0]], 'dataType':'reference'})
			else:
				tana_related_highliights.append({'name':title2})

		if False:
		
			rank = 1
			for hid in semantic_recommendations.loc[highlight_id, "Same Author Recs"]:
				#print(hid)
				title2, score = get_title_for_highlight_id(hid)
				atext = "- sameauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
				rank +=1

				if hid in highlight_id_dict.keys():
					tana_related_highliights.append({'id':highlight_id_dict[hid], 'dataType':'reference'})

				else:
					tana_related_highliights.append({'name':title2})
			
			rank = 1
			for hid in semantic_recommendations.loc[highlight_id, "Diff Author Recs"]:
				title2, score = get_title_for_highlight_id(hid)
				atext = "- diffauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
				rank +=1

				if hid in highlight_id_dict.keys():
					tana_related_highliights.append({'id':highlight_id_dict[hid], 'dataType':'reference'})
				else:
					tana_related_highliights.append({'name':title2})
			
			
			rank = 1
			for hid in concept_based_recommendations.loc[highlight_id, "Same Author Recs"]:
				#print(hid)
				title2, score = get_title_for_highlight_id(hid)
				atext = "- sameauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
				rank +=1
			

			rank = 1
			for hid in concept_based_recommendations.loc[highlight_id, "Diff Author Recs"]:
				#print(hid)
				title2, score = get_title_for_highlight_id(hid)
				atext = "- diffauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
				rank +=1

		
		active_concepts = []
		
		for concept_row in mentions_df[mentions_df["Text_ID"]==highlight_id].itertuples():
			concept_name = concept_row[9]
			concept_url = concept_row[10]
			if concept_name not in active_concepts and concept_name in list(concept_id_dict.keys()):
				
				active_concepts.append(concept_name)



		print("RELATED:  ", tana_related_highliights)

		highlight_concepts_field = []

		for concept_name in active_concepts:
			concept_id = concept_id_dict[concept_name]
			highlight_concepts_field.append({'id':concept_id, 'dataType':'reference'})

		if len(highlight_concepts_field) <= 0:
			highlight_data = {
					'nodes': [
					{
					'name': title,
					'supertags': [{'id':highlight_tag}],
					'children': [
						{
						'type': 'field',
						'attributeId': highlight_author,
						'children': [
						{
							'id': author_id_dict[author],
							'dataType': 'reference'
						},]
						},
						{
						'type': 'field',
						'attributeId': highlight_book,
						'children': [
						{
							'id': book_id_dict[book],
							'dataType': 'reference'
						}]
						},
						{
						'type': 'field',
						'attributeId': highlight_text,
						'children': [
						{
							'name': text
							#'supertags': [{'id':text_tag}]
						},
						]},
						{
						'type': 'field',
						'attributeId': nearest_highlights,
						'children': tana_related_highliights
						}
					]
					}
					]
				}
		else:
			print("HAS CONCEPTS ...")
			highlight_data = {
				'nodes': [
				{
				'name': title,
				'supertags': [{'id':highlight_tag}],
				'children': [
					{
					'type': 'field',
					'attributeId': highlight_author,
					'children': [
					{
						'id': author_id_dict[author],
						'dataType': 'reference'
					},]
					},
					{
					'type': 'field',
					'attributeId': highlight_book,
					'children': [
					{
						'id': book_id_dict[book],
						'dataType': 'reference'
					}]
					},
					{
					'type': 'field',
					'attributeId': highlight_text,
					'children': [
					{
						'name': text
					},
					]},
					{
					'type': 'field',
					'attributeId': highlight_concepts,
					'children': highlight_concepts_field
					},
					{
					'type': 'field',
					'attributeId': nearest_highlights,
					'children': tana_related_highliights
					}
				]
				}
				]
			}
	
		print("POSTING... " + title)
		
		print(highlight_data)

		try:
		
			r = requests.post(endpoint, json=highlight_data, headers=headers)
			
			posted_2 = r.json()

			if 'children' in posted_2.keys():
				highlight_id_dict[highlight_id] = posted_2['children'][0]['nodeId']

		except:
			print("ERROR, didn't post")
			pass
		
		print(posted_2)

"""
{
'type': 'field',
'attributeId': highlight_author,
'children': [
{
	'id': author_id_dict[author],
	'dataType': 'reference'
},]
},
{
'type': 'field',
'attributeId': highlight_book,
'children': [
{
	'id': book_id_dict[book],
	'dataType': 'reference'
}]
},
"""

Sönke Ahrens
{'children': [{'nodeId': 'f91zDpZ2EGyh', 'name': 'Sönke Ahrens', 'type': 'node', 'children': [{'nodeId': 'P-kZYdaX8ws3', 'name': 'Sönke Ahrens', 'type': 'node'}]}]}
f91zDpZ2EGyh
POSTING... How to Take Smart Notes
{'nodes': [{'name': 'How to Take Smart Notes', 'supertags': [{'id': 'UDLQyNUHvs'}], 'children': [{'type': 'field', 'attributeId': 'VmnhTQRb5d', 'children': [{'id': 'f91zDpZ2EGyh', 'dataType': 'reference'}]}]}]}
{'children': [{'nodeId': 'cnrDdTK2xuUp', 'name': 'How to Take Smart Notes', 'type': 'node', 'children': [{'type': 'reference', 'nodeId': 'f91zDpZ2EGyh'}]}]}
{290432993: 0.91128}
TESTEEE
NAME:  How to Take Smart Notes Note 6
TESTEEE NOOOOO
{290432995: 0.5151}
TESTEEE
NAME:  How to Take Smart Notes Note 8
TESTEEE NOOOOO
RELATED:   [{'name': 'How to Take Smart Notes Note 6'}, {'name': 'How to Take Smart Notes Note 8'}, {'name': 'Building a Second Brain Note 32'}, {'name': 'Building a Second Brain Note 7'}]
HAS CONCEPTS ...
POSTING... How to Take Smart Notes No

"\n{\n'type': 'field',\n'attributeId': highlight_author,\n'children': [\n{\n\t'id': author_id_dict[author],\n\t'dataType': 'reference'\n},]\n},\n{\n'type': 'field',\n'attributeId': highlight_book,\n'children': [\n{\n\t'id': book_id_dict[book],\n\t'dataType': 'reference'\n}]\n},\n"

In [232]:
list(hid.keys())

[583516123]

In [233]:
"""for concept_url in concepts_list:
	
	name = concepts_df.loc[concept_url, "Name"]
	image_url = concepts_df.loc[concept_url, "Image"]
	abstract = concepts_df.loc[concept_url, "Abstract"]

	create_concept_md_file(name, concept_url, abstract, image_url, concept_links)
"""

'for concept_url in concepts_list:\n\t\n\tname = concepts_df.loc[concept_url, "Name"]\n\timage_url = concepts_df.loc[concept_url, "Image"]\n\tabstract = concepts_df.loc[concept_url, "Abstract"]\n\n\tcreate_concept_md_file(name, concept_url, abstract, image_url, concept_links)\n'